## CAS Deep Learning - Computer Vision mit Deep Learning (Part 1)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Neuronale Netzwerke

## Lernziele

- Neuronale Netzwerke (MLP): Definieren, Trainieren und Visualisieren
- Anwendung und Interpretation auf Bildern


Hier gibt es gute Videos um mit Neuronalen Netzwerken vertraut zu werden, falls Sie ihr Wissen auffrischen möchten:

[3blue1brown: Aber was *ist* nun ein neuronales Netzwerk?](https://youtu.be/aircAruvnKk?feature=shared) - Es gibt 3 Teile.


Um die folgenden Aufgaben zu bewältigen können Sie das folgende Tutorial zu Hilfe nehmen:

[PyTorch Tutorial Building Model](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)

## Setup

Im Folgenden installieren und laden wir die benötigten Python packages. Danach setzten wir die Pfade für den Zugriff auf Daten und spezifizieren einen Output-Folder.

In [ ]:
import os
from pathlib import Path

Mount your google drive to store data and results.

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print(f"In colab: {IN_COLAB}")

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

Modifizieren Sie die folgenden Pfade bei Bedarf.

In [ ]:
if IN_COLAB:
    DATA_PATH = Path('/content/drive/MyDrive/bverI/data')
else:
    DATA_PATH = Path('../data')

Install packages not in base Colab environment.

In [ ]:
if IN_COLAB:
    os.system("pip install torchshow torchinfo gdown")

## Datensatz: CIFAR 10

Wir bereiten nun den CIAR10 Datensatz vor. Wir verwenden dazu [`torchvision.datasets`](https://pytorch.org/vision/0.8/datasets.html) um ein Objekte der Klasse [`torch.utils.data.Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) zu erstellen.

Später verwenden wir auch den [`torch.utils.data.DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader).


Erstellen Sie ein `torch.utils.data.Dataset` mit Hilfe von `torchvision.datasets.CIFAR10`.

In [ ]:
import torchvision
from  torchvision import transforms

# Transform data to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.5,), (0.5,))
    torchvision.transforms.Lambda(lambda x: torch.nn.functional.normalize(x))
])

# Load CIFAR-10 dataset
training_data = torchvision.datasets.CIFAR10(
    root=DATA_PATH, train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10(
    root=DATA_PATH, train=False, download=True, transform=transform)

classes = training_data.classes

## Lineares Modell

In der folgenden Aufgabe implementieren Sie ein lineares Modell. Danach versuchen Sie den `CIFAR10` Datensatz damit zu modellieren.


Definieren Sie eine Klasse, die von `torch.nn.Module` erbt und definieren Sie Ihr lineares Modell. Es gibt $k=10$ Klassen, wobei jedes Bild $p=32 \times 32 \times 3$ Inputs hat.


Verwenden Sie folgende Klasse um den linearen Layer zu implementieren: [torch.nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
- Verwenden Sie *keinen* Bias-Term, bzw. setzen Sie diesen auf 0

Da dieses Modell als Input nur 1-D Vektoren (oder Batches von solchen Vektoren) prozessieren kann müssen Sie Bilddaten _flach_ drücken / reshapen. Dies können Sie mit:

- [torch.nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) -> Klasse mit `forward()` Methode
- oder mit `x.view(batch, -1)`
- oder mit [torch.flatten](https://pytorch.org/docs/stable/generated/torch.flatten.html) -> Funktion

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo

class LinearModel(nn.Module):

    def __init__(self):
        super().__init__()
        # self.linear_layer = (...) mit torch.nn.Linear
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def forward(self, x):
        # 1) flatten Sie den Input
        # 2) Transformieren Sie den Input mit dem linearen Layer
        # YOUR CODE HERE
        raise NotImplementedError()

net = LinearModel()
print(net)
print(torchinfo.summary(net, input_size=(1, 3, 32, 32)))

**Frage** Wieviele Parameter hat das Modell und warum?

Nun erstellen wir eine Funktion die ein Modell über eine Epoche, d.h. alle Trainingsdaten genau einmal, optimiert. 

In [ ]:
from tqdm.notebook import tqdm 

torch.manual_seed(123)

def train_one_epoch(dataloader, net, optimizer, loss_fn):
    
    with tqdm(dataloader, unit="batch") as tepoch:
        correct_epoch = 0
        total_epoch = 0
        for i, (X, y) in enumerate(tepoch):

            # Compute prediction and loss
            logits = net(X)
            probs = torch.softmax(logits, axis=-1)
            loss = loss_fn(logits, y)
            

            # Backpropagation und Weight Updates
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute Batch Metric
            y_hat = probs.argmax(dim=1, keepdim=True).squeeze()
            correct = (y_hat == y).sum().item()
            accuracy = correct / X.shape[0]

            # Compute Epoch Metric
            correct_epoch += correct
            total_epoch += X.shape[0]
            accuracy_epoch = correct_epoch / total_epoch

            tepoch.set_postfix(loss=loss.item(), accuracy_batch= accuracy * 100, accuracy_epoch = accuracy_epoch * 100)

Nun definieren wir einen DataLoader, Loss-Funktion, sowie Optimizer.


Wichtig: Die Lossfunktion [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) erwartet als Input die `logits`, also keine Wahrscheinlichkeiten. Sie sollen also keine _Softmax_ Funktion anwenden!

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(training_data, batch_size=512, shuffle=True, drop_last=False)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

Nun trainieren wir das Modell für 5 Epochen. Verwenden Sie dazu die Funktion `train_one_epoch()`.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Was hat das lineare Modell gelernt?

In dieser Aufgabe visualisieren Sie was das Modell gelernt hat. Zeichnen Sie dazu die Gewichte $\mathbf{W} \in \mathbb{R}^{k \times p}$ als Bilder.

Dabei hat das Modell für jede der $k$ Klassen jeweils $p$ Gewichte gelernt.



In [ ]:
from matplotlib import pyplot as plt

weights = net.linear_layer.weight.data.cpu()
weights = weights.numpy()

fig, axes = plt.subplots(2, 5, figsize=(20, 10))
axes = axes.ravel()

for i, ax in enumerate(axes):

    # Reshape the weights into images
    # weight = weights[i].(...)
    # YOUR CODE HERE
    raise NotImplementedError()
    
    weight = (weight - weight.min()) / (weight.max() - weight.min())
    _ = ax.imshow(weight)
    _ = ax.set_title(classes[i])
    _ = ax.axis('off')
plt.show()

Wie interpretieren Sie die Gewichte? Warum sehen die so aus?

YOUR ANSWER HERE

## Neuronales Netzwerk

In dieser Aufgabe werden Sie ein neuronales Netzwerk mit einem Hidden-Layer trainieren.

Ergänzen Sie die Klasse `MLP` und definieren Sie Ihr neuronales Netzwerk. Erstellen Sie einen Hidden-Layer mit 64 Nodes und ReLU Aktivierungs-Funktion. Instanzieren Sie das Netzwerk und printen Sie das Objekt.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo

class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def forward(self, x):
        # YOUR CODE HERE
        raise NotImplementedError()
        return x


net = MLP()
print(torchinfo.summary(net, input_size=(1, 3, 32, 32)))

**Frage** Wieviele Parameter hat das Modell und warum?

Trainieren Sie das Modell für 3-10 Epochen.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Visualisieren Sie die Gewichte im ersten Layer.

In [ ]:
# selektieren Sie die Gewichte
import numpy as np

# weights = net.(...)
# YOUR CODE HERE
raise NotImplementedError()

# import torchshow as ts
# ts.show(weights)

weights = weights.numpy()

fig, axes = plt.subplots(8, 8, figsize=(10, 10))
axes = axes.ravel()

for i, ax in enumerate(axes):

    # Reshape the weights into images
    weight = weights[i].reshape(3, 32, 32).transpose(1, 2, 0)
    weight = (weight - weight.min()) / (weight.max() - weight.min())
    _ = ax.imshow(weight)
    _ = ax.axis('off')

plt.tight_layout()
plt.show()

Wie interpretieren Sie die Visualisierungen?

YOUR ANSWER HERE

## (Optional) Ausblick CNNs

- Implementieren Sie ein CNN.
- Trainieren Sie das Modell.
- Vergleichen Sie die Performance.
- Visualisieren Sie die gelernten Filter.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo

class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError()
        
    def forward(self, x):
        # YOUR CODE HERE
        raise NotImplementedError()
        return x


cnn = CNN()
print(torchinfo.summary(cnn, input_size=(1, 3, 32, 32)))

**Frage** Wieviele Parameter hat das Modell und warum?

In [ ]:
dataloader = DataLoader(training_data, batch_size=512, shuffle=True, drop_last=False)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters())

for epoch in range(0, 5):
    train_one_epoch(dataloader, cnn, optimizer, loss_fn)


In [ ]:
# selektieren Sie die Gewichte
import numpy as np

# weights = cnn.(...)
# YOUR CODE HERE
raise NotImplementedError()

weights = weights.numpy()

fig, axes = plt.subplots(4, 4, figsize=(5, 5))
axes = axes.ravel()

for i, ax in enumerate(axes):

    # Reshape the weights into images
    weight = weights[i].reshape(3, 7, 7).transpose(1, 2, 0)
    weight = (weight - weight.min()) / (weight.max() - weight.min())
    _ = ax.imshow(weight)
    _ = ax.axis('off')

plt.tight_layout()
plt.show()